In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from pyts.datasets import make_cylinder_bell_funnel

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
random_state = 0

# IMPORT DATASET

In [4]:
dataset_name = "cbf"

In [5]:
X_all, y_all = make_cylinder_bell_funnel(n_samples = 600, random_state = random_state)
X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], 1))

In [6]:
print("X SHAPE: ", X_all.shape)
print("y SHAPE: ", y_all.shape)
unique, counts = np.unique(y_all, return_counts=True)
print("\nCLASSES BALANCE")
for i, label in enumerate(unique):
    print(label, ": ", round(counts[i]/sum(counts), 2))

X SHAPE:  (600, 128, 1)
y SHAPE:  (600,)

CLASSES BALANCE
0 :  0.33
1 :  0.33
2 :  0.33


## DATASET SPLIT

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# BLACKBOX/EXPLANATION SETS SPLIT
X_train, X_exp, y_train, y_exp = train_test_split(X_all, y_all, 
                                                  test_size=0.3, stratify = y_all, random_state=random_state)

# BLACKBOX TRAIN/TEST SETS SPLIT
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# BLACKBOX TRAIN/VALIDATION SETS SPLIT
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# EXPLANATION TRAIN/TEST SETS SPLIT
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, 
                                                                    test_size=0.2, 
                                                                    stratify = y_exp, 
                                                                    random_state=random_state)

# EXPLANATION TRAIN/VALIDATION SETS SPLIT
X_exp_train, X_exp_val, y_exp_train, y_exp_val = train_test_split(X_exp_train, y_exp_train, 
                                                                  test_size=0.2, 
                                                                  stratify = y_exp_train, 
                                                                  random_state=random_state)

print("SHAPES:")
print("BLACKBOX TRAINING SET: ", X_train.shape)
print("BLACKBOX VALIDATION SET: ", X_val.shape)
print("BLACKBOX TEST SET: ", X_test.shape)
print("EXPLANATION TRAINING SET: ", X_exp_train.shape)
print("EXPLANATION VALIDATION SET: ", X_exp_val.shape)
print("EXPLANATION TEST SET: ", X_exp_test.shape)

SHAPES:
BLACKBOX TRAINING SET:  (268, 128, 1)
BLACKBOX VALIDATION SET:  (68, 128, 1)
BLACKBOX TEST SET:  (84, 128, 1)
EXPLANATION TRAINING SET:  (115, 128, 1)
EXPLANATION VALIDATION SET:  (29, 128, 1)
EXPLANATION TEST SET:  (36, 128, 1)


In [9]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_all)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

TIMESTEPS:  128
N. LABELS:  3


# BLACKBOX MODEL

In [10]:
from joblib import load
import keras
"""import importlib
importlib.reload(blackboxes)"""
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import time
import blackboxes
from blackboxes import *

Using TensorFlow backend.


## resnet

In [11]:
blackbox = build_resnet(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/cbf_blackbox_resnet_20191106_145242_best_weights_+1.00_.hdf5")
resnet = blackbox

## simple CNN

In [12]:
blackbox = build_simple_CNN(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/cbf_blackbox_simpleCNN_20191106_145515_best_weights_+1.00_.hdf5")
simplecnn = blackbox

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [14]:
knn = load(parentdir + "/blackbox_checkpoints/cbf_blackbox_knn_20191106_145654.joblib")

# AUTOENCODER

In [15]:
"""import importlib
importlib.reload(autoencoders)"""
import autoencoders
from autoencoders import *

## VARIATIONAL

In [16]:
params = {"input_shape": (n_timesteps,1),
          "n_blocks": 8, 
          "latent_dim": 2,
          "encoder_latent_layer_type": "variational",
          "encoder_args": {"filters":[2,4,8,16,32,64,128,256], 
                            "kernel_size":[21,18,15,13,11,8,5,3], 
                            "padding":"same", 
                            "activation":"elu", 
                            "pooling":[1,1,1,1,1,1,1,1]}
         }

aut = Autoencoder(verbose = False, **params)
encoder, decoder, autoencoder = aut.build()
autoencoder.load_weights(parentdir + "/autoencoder_checkpoints/cbf_autoencoder_20191106_144909_best_weights_+136.8745_.hdf5")
vae = autoencoder

# GLOBAL VS LOCAL SHAPELET TREE

In [17]:
import agnosticlocalexplainer
from agnosticlocalexplainer import *
import importlib
importlib.reload(agnosticlocalexplainer)

/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


<module 'agnosticlocalexplainer' from '/Users/francesco/github/TS_AgnosticLocalExplainer/agnosticlocalexplainer.py'>

In [18]:
from myutils import BlackboxPredictWrapper
import time
from agnosticglobalexplainer import AgnosticGlobalExplainer
from joblib import dump
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, coverage_error
from global_vs_local_surrogate import shapelet_local_explain

## RESNET

In [19]:
autoencoder = vae
encoder = autoencoder.layers[1]
decoder = autoencoder.layers[2]
blackbox = resnet
blackbox_input_dimensions = 3

In [39]:
blackbox_predict = BlackboxPredictWrapper(blackbox, 3)
args = {"max_iter": 100}

global_surrogate = AgnosticGlobalExplainer(**args)
global_surrogate.fit(X_exp_train[:,:,0], blackbox_predict.predict(X_exp_train))

y_blackbox = blackbox_predict.predict(X_exp_test)
y_blackbox_proba = blackbox_predict.predict_proba(X_exp_test)
y_surrogate = global_surrogate.predict(X_exp_test[:,:,0])
y_surrogate_proba = global_surrogate.predict_proba(X_exp_test[:,:,0])

global_results = {"shapelet_explainer": global_surrogate}

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    3.0s finished


In [21]:
%%time
local_results = shapelet_local_explain(blackbox, 
                           encoder, 
                           decoder, 
                           autoencoder, 
                           X_exp_test,
                           y_exp_test, 
                           blackbox_input_dimensions, 
                           ngen = 10, 
                           size = 1000, 
                           neigh_type = "geneticp",
                           labels = ["cylinder", "bell", "funnel"])


Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.485291	0.00669472	0.993287
2  	424   	0.552972	0.00671962	0.993287
3  	432   	0.643348	0.00673406	0.993303
4  	415   	0.718148	0.0071287 	0.993306
5  	422   	0.747955	0.00841192	0.993306
6  	424   	0.738615	0.00768505	0.993306
7  	409   	0.734051	0.00861209	0.993306
8  	420   	0.742482	0.00741322	0.993307
9  	408   	0.766868	0.00959444	0.993306
10 	412   	0.76297 	0.00839571	0.993306
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	411   	0.499712	0.00669416	0.993218
2  	423   	0.579289	0.00681029	0.993224
3  	422   	0.671219	0.00692127	0.993245
4  	414   	0.717583	0.00708291	0.993286
5  	427   	0.775689	0.00967016	0.993307
6  	417   	0.787485	0.0101663 	0.993307
7  	417   	0.816213	0.00757514	0.993307
8  	432   	0.805098	0.00730904	0.993307
9  	421   	0.832276	0.0232477 	0.993307
10 	420   	0.823119	0.00669684	0.993307
synthetic neighborhood cla

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 181 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.7s finished


1 / 36

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.581497	0.00673318	0.993307
2  	424   	0.745194	0.00673424	0.993307
3  	432   	0.821767	0.00686805	0.993307
4  	415   	0.884993	0.00671182	0.993307
5  	422   	0.890733	0.00707646	0.993307
6  	424   	0.887714	0.0141947 	0.993307
7  	409   	0.868018	0.00969019	0.993307
8  	420   	0.87468 	0.00670058	0.993307
9  	408   	0.86844 	0.00673692	0.993307
10 	412   	0.865972	0.00682341	0.993307
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	411   	0.55645	0.490769	0.993301
2  	423   	0.662643	0.488803	0.993301
3  	422   	0.773712	0.489598	0.993304
4  	414   	0.842539	0.488983	0.993306
5  	427   	0.878369	0.493126	0.993307
6  	417   	0.889569	0.488907	0.993307
7  	417   	0.899287	0.49343 	0.993307
8  	432   	0.893096	0.488922	0.993307
9  	421   	0.903564	0.491659	0.993307
10 	420   	0.8986  	0.489729	0.993307
synthetic neighborhood class counts {0: 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 294 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.0s finished


2 / 36

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.483562	0.00670328	0.993304
2  	424   	0.558538	0.00669748	0.993304
3  	432   	0.666503	0.00670451	0.993305
4  	415   	0.733513	0.00679001	0.993306
5  	422   	0.780176	0.00669517	0.993306
6  	424   	0.776707	0.00675469	0.993306
7  	409   	0.751416	0.00669972	0.993307
8  	420   	0.751651	0.00671137	0.993307
9  	408   	0.788454	0.00669466	0.993307
10 	412   	0.782725	0.00670298	0.993307
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	411   	0.495694	0.00671549	0.993174
2  	423   	0.552465	0.00670535	0.993264
3  	422   	0.643001	0.00670535	0.993307
4  	414   	0.720202	0.00677447	0.993304
5  	427   	0.789951	0.00670493	0.993307
6  	417   	0.81371 	0.00669727	0.993307
7  	417   	0.843913	0.00670905	0.993307
8  	432   	0.826351	0.0069048 	0.993307
9  	421   	0.845607	0.00674388	0.993307
10 	420   	0.844716	0.00671729	0.993307
synthetic neighborh

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 309 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.2s finished


35 / 36

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.491541	0.00669973	0.993307
2  	424   	0.574488	0.00670714	0.993307
3  	432   	0.697406	0.00669978	0.993307
4  	415   	0.750408	0.00677382	0.993307
5  	422   	0.794766	0.00669529	0.993307
6  	424   	0.78367 	0.00671638	0.993307
7  	409   	0.756357	0.00671755	0.993307
8  	420   	0.760247	0.00669341	0.993307
9  	408   	0.793125	0.00672569	0.993307
10 	412   	0.785786	0.00669573	0.993307
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	411   	0.498507	0.00669725	0.993297
2  	423   	0.545076	0.0066934 	0.993305
3  	422   	0.637165	0.00689597	0.993305
4  	414   	0.712597	0.00669444	0.993306
5  	427   	0.789986	0.00697508	0.993306
6  	417   	0.811843	0.00670112	0.993306
7  	417   	0.835768	0.00669751	0.993306
8  	432   	0.828873	0.00697066	0.993307
9  	421   	0.847249	0.00674452	0.993306
10 	420   	0.848857	0.0067001 	0.993306
synthetic neighbor

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


36 / 36
CPU times: user 4h 51min 28s, sys: 15min 20s, total: 5h 6min 48s
Wall time: 4h 27min


[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.6s finished


In [40]:
results = {"y_blackbox_original_LOCAL": local_results["y_blackbox_original"], 
               "y_blackbox_reconstructed_LOCAL": local_results["y_blackbox_reconstructed"], 
               "y_surrogate_original_LOCAL": local_results["y_surrogate_original"], 
               "y_surrogate_reconstructed_LOCAL": local_results["y_surrogate_reconstructed"],
               "y_blackbox_GLOBAL": y_blackbox,
               "y_surrogate_GLOBAL": y_surrogate,
               "y_blackbox_original_LOCAL_proba": local_results["y_blackbox_original_proba"], 
               "y_blackbox_reconstructed_LOCAL_proba": local_results["y_blackbox_reconstructed_proba"], 
               "y_surrogate_original_LOCAL_proba": local_results["y_surrogate_original_proba"], 
               "y_surrogate_reconstructed_LOCAL_proba": local_results["y_surrogate_reconstructed_proba"],
               "y_blackbox_GLOBAL_proba": y_blackbox_proba,
               "y_surrogate_GLOBAL_proba": y_surrogate_proba
               }
results_df = results#pd.DataFrame(results)

local_classification_report = classification_report(results_df["y_blackbox_original_LOCAL"], results_df["y_surrogate_reconstructed_LOCAL"])
global_classification_report = classification_report(results_df["y_blackbox_GLOBAL"], results_df["y_surrogate_GLOBAL"])
reconstrution_classification_report = classification_report(results_df["y_blackbox_original_LOCAL"], results_df["y_blackbox_reconstructed_LOCAL"])

local_fidelity = accuracy_score(results_df["y_blackbox_original_LOCAL"], results_df["y_surrogate_reconstructed_LOCAL"])
global_fidelity = accuracy_score(results_df["y_blackbox_GLOBAL"], results_df["y_surrogate_GLOBAL"])
reconstruction_fidelity = accuracy_score(results_df["y_blackbox_original_LOCAL"], results_df["y_blackbox_reconstructed_LOCAL"])

local_coverage = coverage_error(results_df["y_blackbox_reconstructed_LOCAL_proba"].round(), results_df["y_surrogate_reconstructed_LOCAL_proba"])
global_coverage = coverage_error(results_df["y_blackbox_GLOBAL_proba"].round(), results_df["y_surrogate_GLOBAL_proba"])


In [41]:
print("local fidelity: ", local_fidelity)
print("global fidelity: ", global_fidelity)
print("reconstruction fidelity: ", reconstruction_fidelity)
print("\n")
print("local coverage: ", local_coverage)
print("global coverage: ", global_coverage)
print("\n")
print("local classification report:")
print(local_classification_report)
print("\n")
print("global classification report:")
print(global_classification_report)
print("\n")
print("reconstrution classification report:")
print(reconstrution_classification_report)

local fidelity:  1.0
global fidelity:  0.9722222222222222
reconstruction fidelity:  1.0


local coverage:  1.0
global coverage:  1.0555555555555556


local classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        12

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



global classification report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       1.00      0.92      0.96        12
           2       1.00      1.00      1.00        12

    accuracy                           0.97        36
   macro avg       0.97      0.97      0.97        36
weighted avg       0.97      0.97      0.97        36



reconstrution classificat

In [42]:
file_path = parentdir + "/agnostic_explainers/" + dataset_name + "_agnosticvsglobal_" + time.strftime("%Y%m%d_%H%M%S")
results_all = {"local_results": local_results,
               "global_results": global_results,
               "results_y": results   
}
dump(results_all, file_path + ".joblib")


['/Users/francesco/github/TS_AgnosticLocalExplainer/agnostic_explainers/cbf_agnosticvsglobal_20191211_181444.joblib']

In [44]:
dump(local_results, file_path + "_local_results.joblib")
dump(global_results, file_path + "_global_results.joblib")
dump(results, file_path + "_results_y.joblib")

['/Users/francesco/github/TS_AgnosticLocalExplainer/agnostic_explainers/cbf_agnosticvsglobal_20191211_181444_results_y.joblib']

In [ ]:
"""dump(local_results, file_path + "_local_results.joblib")
#dump(global_results, file_path + "global_results.joblib")
dump(results, file_path + "_results_y.joblib")"""